### Figure Friday 2024 - W49
In this week’s Figure-Friday we’ll look at the hourly demand for electricity in New England, US, provided by the US Energy Information Administration (EIA). In case you’re interested in additional data sets for the New England regions or would like to explore the other graphs made by the EIA, visit their wholesale markets page.</br>

**Data filtering overview**</br>
We are used to click on pictures, links or any other clickable things we stumbled upon, so I focused when building this Plotly app, on **chaining callbacks** allowing the user to update the 2nd bar chart accordingly to the selected state from the previous 1st bar chart. In the example the idea is ‘go deeper’ (one level down into the details) to explore data or find some insights the chart shows us.


In [ ]:
"""Just importing"""
from dash import Dash, html, Output, Input, callback, dcc, no_update, ctx
import dash_vtk
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import pandas as pd
import plotly.express as px
import plotly.io as pio
import pprint

# print(pio.templates.default)
pio.templates.default = 'plotly_white'

# loads the "bootstrap" template and sets it as the default
# load_figure_template("bootstrap")

# importing and preparing the data ++++++++++++++++++++++++++++++++++++++
data = pd.read_csv("https://raw.githubusercontent.com/plotly/Figure-Friday/refs/heads/main/2024/week-49/megawatt_demand_2024.csv")

# Convert the "Local Timestamp" column to a datetime format and filter for October data
data['Local Timestamp'] = pd.to_datetime(data['Local Timestamp Eastern Time (Interval Beginning)'])
october_data = data[(data['Local Timestamp'] >= '2024-10-01') & (data['Local Timestamp'] < '2024-11-01')]

# Prepare the regional data for plotting
regions = [
    "Connecticut Actual Load (MW)", "Maine Actual Load (MW)",
    "New Hampshire Actual Load (MW)", "Northeast Massachusetts Actual Load (MW)",
    "Rhode Island Actual Load (MW)", "Southeast Massachusetts Actual Load (MW)",
    "Vermont Actual Load (MW)", "Western/Central Massachusetts Actual Load (MW)"
]
df = data.select_dtypes(include=['floating', 'datetime']).copy()
df.set_index('Local Timestamp', inplace=True) # for resampling


# renaming columns for shorter displays cols
old_values = df.columns.values.tolist()
new_values = [elem[:-17] for elem in old_values]
mapper = {old:new for old, new in zip(old_values, new_values)}
df.rename(mapper=mapper, axis=1, inplace=True)

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

radio_group = dmc.RadioGroup(
    id= 'radiogroup_period',
    label='Select period to aggregate',
    value=[],
    size='sm',
    # mb=5,
    children= dmc.Group([
        dmc.Radio(label='Year', value='YE'),
        dmc.Radio(label='Quarter', value='QE'),
        dmc.Radio(label='Month', value='ME'),
    ], className='m-2'), className='border ps-2 my-1',
)

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H3("New England energy usage",
                        className="text-start text-primary my-3"),
                width=11)
    ], justify='around'), #align='center', 
    dbc.Row([
        dbc.Col(radio_group, width=6),
        dbc.Col(html.H5('Radar Chart',
                        className='text-center text-primary my-2'),
                width=5)
    ], align='center', justify='around'),
    # dbc.Row([dmc.Text(id="radio_output")]), # added to see clickData, for debugging callback update_txt
    dbc.Row([
        dbc.Col(dcc.Graph(id="bar_chart_1", figure={}, className='shadow rounded'), width=6),
        dbc.Col(dcc.Graph(id="radar_chart", figure={}, className='shadow rounded'), width=5),
    ], justify='around'), #align='center', 
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    dbc.Row([
        dbc.Col(html.H5('Bar Group - 1 level down on hover click',
                        className='text-center text-primary my-2'),
                width=6),
        dbc.Col(html.H5('Line chart - 1 level down on hover click',
                        className='text-center text-primary my-2'),
                width=5)
    ], align='center', justify='around'), # align='center'
    dbc.Row([
        dbc.Col(dcc.Graph(id="bar_chart_2", figure={}, className='shadow rounded'), width=6),
        dbc.Col(dcc.Graph(id="line_chart", figure={}, className='shadow rounded'), width=5),
    ], justify='around'), #align='center', 
], fluid=True)


# # For debugging clickData ++++++++++++++++++++++++++++++
# @callback(
#         Output("radio_output", "children"),
#         Input('bar_chart_1', 'clickData')
# )
# def update_txt(value):
#     return pprint.pformat(value)
# # ++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Update bar_chart_1 & radar_chart => value=['YE', 'QE', 'ME']
@callback(
    Output("bar_chart_1", "figure"),
    Output('radar_chart', 'figure'),
    Input("radiogroup_period", "value"),
    prevent_initial_call = True
)
def update_bar_chart(value):

    resample = (df
                .resample(value)
                .sum()
    )
    res_melted = (resample
                  .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                  .reset_index()
                  .sort_values(by='Load_MW')
    )
    fig_bar = px.bar(res_melted, x='Load_MW', y='state', color='Local Timestamp',
              labels={'state':''}, height=450,
              title =f'Energy usage by Y2024 - {value}',
              template='plotly_white', color_discrete_sequence= px.colors.qualitative.D3,
             )
    fig_bar.update_layout(showlegend=False)

    fig_bar_polar = px.bar_polar(
                    res_melted,
                    r='Load_MW', theta='state',
                    color='Local Timestamp',
                    width=700, template='plotly_white',
                    color_discrete_sequence= px.colors.qualitative.D3
    )
    fig_bar_polar.update_layout(showlegend=False)

    return fig_bar, fig_bar_polar

# Update bar_chart_2 according to clickData from bar_chart_1 and value from radiogroup_period
@callback(
    Output('bar_chart_2', 'figure'),
    Output('line_chart', 'figure'),
    Input("radiogroup_period", "value"),
    Input('bar_chart_1', 'clickData'), ## ++++ clickData
    prevent_initial_call = True
    # running=[(Output('bar_chart_2', 'figure'), fig_bar_2, {})]
)
def update_bar_chart_2(value, clickData):
    # Printing some values to check data collected
    # print(value)
    # print(type(clickData['points'][0]['y']))
    # print(f"Click Data:\n{clickData['points'][0]['y']}" if clickData else "No click data")
    # print(f"Click Data:\n{clickData['points'][0]}" if clickData else clickData)
    # print('the input: ', ctx.triggered_id)

    # df resample by 'Day' for further usage
    df_day = (df.resample('D').sum())
    df_day_melted = (df_day
                     .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                     .reset_index()
                     .sort_values(by='Local Timestamp')
                     )
    df_qe = (df.resample('QE').sum())
    dff_qe = (df_qe
                .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                .reset_index()
                .sort_values(by='Load_MW')
    )
    df_me = (df.resample('ME').sum())
    dff_me = (df_me
                .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                .reset_index()
                .sort_values(by='Load_MW')
    )
    df_bar2 = (df.resample('W').sum())
    dff_bar2 = (df_bar2
                .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                .reset_index()
                .sort_values(by='Load_MW')
    )
    
    if ctx.triggered_id == 'bar_chart_1':   # to verify which input triggered the callback !!!

        if clickData and value == 'YE': # Resample by Quarter to bar_chart_2 and px_line
            var_state = clickData['points'][0]['y']
            
            dff_bar2_filtered = dff_qe[dff_qe['state']== var_state]
            fig_bar_2 = px.bar(dff_bar2_filtered, y='Load_MW', x='state', color=dff_bar2_filtered['Local Timestamp'].dt.strftime('%Y-%m'),
                    labels={'state':''}, height=450,
                    title = 'Energy usage by Quarter',
                    template='plotly_white', color_discrete_sequence= px.colors.qualitative.D3,
                    barmode='group',
            )
            temp = df_day_melted[df_day_melted['state'] == var_state]
            fig_line = px.line(temp, x='Local Timestamp', y='Load_MW',
                            template='plotly_white', labels={'Local Timestamp':var_state})
            
            return fig_bar_2, fig_line
            
        elif clickData and value == 'QE':
            var_state = clickData['points'][0]['y']
            load_to_lookup = clickData['points'][0]['x']
            # print(load_to_lookup)

            temp_3 = dff_qe[dff_qe['state']== var_state]
            temp_4 = temp_3[temp_3['Load_MW'] == load_to_lookup]
            end_m = temp_4['Local Timestamp'].dt.strftime('%Y-%m').values[0]
            to_rep = temp_4['Local Timestamp'].dt.strftime('%Y-%m').values[0][-2:]

            if to_rep != '12':
                start_m = end_m[:-2]+'0'+str(int(to_rep)-2)# >> '09'-'06'-'03'
            else: start_m = end_m[:-2]+str(int(to_rep)-2)# >> '12'

            df2_bar2_ff = df_me[start_m:end_m]
            dff_bar2 = (df2_bar2_ff
                        .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                        .reset_index()
                        .sort_values(by='Load_MW')
            )
            dff_bar2_filtered = dff_bar2[dff_bar2['state']== var_state]
            fig_bar_2 = px.bar(dff_bar2_filtered, y='Load_MW', x='state', color=dff_bar2_filtered['Local Timestamp'].dt.strftime('%Y-%m'),
                    labels={'state':''}, height=450,
                    title = 'Energy usage by Month in sel Q',
                    template='plotly_white', color_discrete_sequence= px.colors.qualitative.D3,
                    barmode='group',
            )
            dff_day = df_day[start_m:end_m]
            df_day_melted = (dff_day
                            .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                            .reset_index()
                            .sort_values(by='Local Timestamp')
                            )
            temp = df_day_melted[df_day_melted['state'] == var_state]
            fig_line = px.line(temp, x='Local Timestamp', y='Load_MW',
                            template='plotly_white', labels={'Local Timestamp':var_state})
            
            return fig_bar_2, fig_line
            
        elif clickData and value == 'ME':   # Month resample
            var_state = clickData['points'][0]['y']
            load_to_lookup = clickData['points'][0]['x']

            temp_3 = dff_me[dff_me['state']== var_state]
            temp_4 = temp_3[temp_3['Load_MW'] == load_to_lookup]
            end_m = temp_4['Local Timestamp'].dt.strftime('%Y-%m').values[0]

            df2_bar2_ff = df_day.loc[end_m]
            dff_bar2 = (df2_bar2_ff
                        .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                        .reset_index()
                        .sort_values(by='Load_MW')
            )
            temp = dff_bar2[dff_bar2['state']== var_state].sort_values(by='Local Timestamp').copy()
            fig_bar_2 = px.bar(temp, y='Load_MW', x='state', color=temp['Local Timestamp'].dt.strftime('%b, %d'),
                    labels={'state':''}, height=450,
                    title = 'Energy usage by Day in sel Month',
                    template='plotly_white', color_discrete_sequence= px.colors.qualitative.D3,
                    barmode='group',
            )
            dff_day = df.loc[end_m]
            dff_day_melted = (dff_day
                            .melt(var_name='state', value_name='Load_MW', ignore_index=False)
                            .reset_index()
                            .sort_values(by='Local Timestamp')
                            )
            ### # temp['Local Timestamp'].dt.strftime('%H:00') tickvals
            temp = dff_day_melted[dff_day_melted['state'] == var_state]
            fig_line = px.line(temp, x='Local Timestamp', y='Load_MW',
                            template='plotly_white', labels={'Local Timestamp':var_state},
                            title='Energy hourly-usage by sel Month')
            return fig_bar_2, fig_line
            
        else: return no_update, no_update
    else: return no_update, no_update


if __name__ == "__main__":
    app.run(debug=True, port=9999, jupyter_mode='external')
